# Funciones SQL

In [ ]:
import org.apache.spark.sql.SparkSession

In [ ]:
val spark = SparkSession.builder.getOrCreate()

In [ ]:
import spark.implicits._

In [ ]:
import org.apache.spark.sql.types._

In [ ]:
val MesaElectoralSchema = StructType(Seq(
    StructField("Distrito", IntegerType, false),
    StructField("Barrio", IntegerType, false),
    StructField("Seccion", IntegerType, false),
    StructField("Mesa", StringType, false),
    StructField("Censo", IntegerType, false),
    StructField("Abstencion", IntegerType, true),
    StructField("Nulos", IntegerType, true),
    StructField("Blanco", IntegerType, true),
    StructField("Emitidos", IntegerType, false))) 

In [ ]:
object StringUtils {
    implicit class StringImprovements(val s:String) {
        import scala.util.control.Exception._
        def toIntSafe = catching(classOf[NumberFormatException]) opt s.toInt
    }
}

In [ ]:
import StringUtils._

In [ ]:
import org.apache.spark.sql.Row

In [ ]:
def stringToRow(row:String):Row = {
    val x = row.split(";")
    Row(x(0).toInt, x(1).toInt, x(2).toInt, x(3).slice(0, 1), x(4).toInt, 
                    x(5).toIntSafe.getOrElse(null), x(7).toIntSafe.getOrElse(null), x(9).toIntSafe.getOrElse(null), 
                    (x(11).toInt + x(12).toInt + x(13).toInt + x(14).toInt +
                    x(15).toInt + x(16).toInt + x(17).toInt + x(18).toInt + x(19).toInt +
                    x(20).toInt + x(21).toInt + x(22).toInt + x(23).toInt + x(24).toInt +
                    x(25).toInt + x(26).toInt + x(27).toInt + x(28).toInt + x(29).toInt +
                    x(30).toInt + x(31).toInt + x(32).toInt))
}

In [ ]:
val elecciones_2015_rdd = sc.textFile("hdfs:///eoi/s4/elecciones/Elecciones_2015_NH.csv")

In [ ]:
val elecciones_2015 = elecciones_2015_rdd.map(r => stringToRow(r))

In [ ]:
val elecciones_2015_DF = spark.createDataFrame(elecciones_2015, MesaElectoralSchema).cache

In [ ]:
elecciones_2015_DF.show

In [ ]:
elecciones_2015_DF.printSchema

In [ ]:
elecciones_2015_DF.columns

In [ ]:
elecciones_2015_DF.dtypes

In [ ]:
import org.apache.spark.sql.functions._

# Ejemplo de funciones `Math`

In [ ]:
val res = elecciones_2015_DF.select($"Distrito", $"Barrio", $"Seccion", 
                                    round(($"Emitidos" / $"Censo"),4) as "Participacion")

In [ ]:
res.show

# Ejemplo de funciones `Sort`

In [ ]:
val res_2 = res.sort(asc("Participacion"))

In [ ]:
res_2.show

In [ ]:
val res_2 = res.sort(asc("Participacion"))

In [ ]:
res_2.show

# Ejemplo de funciones `String`

In [ ]:
val res = elecciones_2015_DF.select($"Distrito", $"Barrio", $"Seccion", $"Mesa").head.getString(3)

In [ ]:
val res = elecciones_2015_DF.select($"Distrito", $"Barrio", $"Seccion", lower($"Mesa")).head

# Ejemplo de funciones `Date-time`

In [ ]:
val aux = elecciones_2015_DF.withColumn("Fecha", typedLit(1432418400))

In [ ]:
aux.show()

In [ ]:
val res = aux.withColumn("Fecha", from_unixtime($"Fecha"))

In [ ]:
res.show

In [ ]:
val res = aux.withColumn("Fecha", from_unixtime($"Fecha", "dd/MM/YYYY hh:mm:ss"))

In [ ]:
res.show

# Ejemplo de funciones de agregación

In [ ]:
val res = elecciones_2015_DF.select($"Distrito", $"Barrio", $"Seccion", 
                                    round(($"Emitidos" / $"Censo"),4) as "Participacion")

In [ ]:
val res_2 = res.select(avg($"Participacion"), stddev($"Participacion"), min($"Participacion"), max($"Participacion"))

In [ ]:
res_2.show

In [ ]:
val res_3 = res.groupBy($"Barrio").agg(avg($"Participacion"), stddev($"Participacion"), min($"Participacion"), max($"Participacion"))


In [ ]:
res_3.show

# Ejemplo de funciones `Window`

In [ ]:
import org.apache.spark.sql.expressions.Window

In [ ]:
val res = elecciones_2015_DF.filter($"Distrito" === 2).select($"Barrio", $"Seccion", $"Mesa", max($"Censo")
.over(Window.partitionBy($"Barrio", $"Seccion")) as "Max Censo Distrito")

In [ ]:
res.show

In [ ]:
val res = elecciones_2015_DF.filter($"Distrito" === 2).select($"Barrio", $"Seccion", $"Mesa", $"Censo", lag($"Censo", 1, -1)
.over(Window.partitionBy($"Barrio", $"Seccion").orderBy($"Barrio", $"Seccion", $"Mesa")) as "Anterior")
.sort($"Barrio", $"Seccion", $"Mesa")

In [ ]:
res.show

# Ejemplo de funciones `UDF`

In [ ]:
val bajaParticipacion = udf((participacion:Double) => if (participacion < 0.25) {true} else {false})

In [ ]:
val altaParticipacion = spark.udf.register("Alta Participacion", 
                                           (participacion:Double) => if (participacion > 0.85) {true} else {false})

In [ ]:
val res = elecciones_2015_DF.select($"Distrito", $"Barrio", $"Seccion", 
                                    round(($"Emitidos" / $"Censo"),4) as "Participacion",
                                    bajaParticipacion(($"Emitidos" / $"Censo")) as "Baja Participacion")

In [ ]:
res.sort(desc("Participacion")).show

In [ ]:
val res = elecciones_2015_DF.select($"Distrito", $"Barrio", $"Seccion", 
                                    round(($"Emitidos" / $"Censo"),4) as "Participacion",
                                    altaParticipacion(($"Emitidos" / $"Censo")) as "Alta Participacion")

In [ ]:
res.sort(desc("Participacion")).show